### Instructions: Follow the instructions provided in each step, or in the output from a cell 
Step 1
* Make sure the python notebook and the pivot table Excel file are in the **same** folder on your computer.
* Enter the name of the pivot table in the next cell, then press the 'run' button above.
* The python notebook will give you a preview of what will be used in the analysis beneath the cell after you press the run button.  
  * Scan the row names to verify that they are the same as in the Excel sheet that you want ot use.  If they are, skip the following cells and proceed to Step 2.
* Otherwise, follow the instructions in the output.

In [1]:
pivot_table = '20411_Felber1-2_MHC-I_Haplotypes_23Mar18.xlsx' # enter name here

In [2]:
# Developer Comment: JRC
# see dev_comments3_beta_1_0.md

In [3]:
# Developer Comment: JRC
# Remaining ToDo:
# # add in MHC-B to ML module
# # remove all developer comments from beta version
# # writeup features that will be added (replace patch, write into pivot_table and not a csv file)
# # final testing

In [9]:
import pandas as pd
from pandas import DataFrame
import re
import time
import math
import numpy as np

#PRAGMA MARK: parseInputData 
# import input data, then parse into Pandas DataFrame
def parseExcelWithPandas(fName, excelFileP):
    eSheetData = ''
    dfs = {sheet_name: excelFileP.parse(sheet_name)
      for sheet_name in excelFileP.sheet_names}
    if not fName:
        sheetCount = -1
        for s in dfs:
            sheetCount += 1
            m = re.search('pivot', s)
            m2 = re.search('MiSeq', s)
            if m:
                eSheetDataInt = pd.ExcelFile(pivot_table)
                eSheetData = eSheetDataInt.parse(sheetCount)
            elif m2:
                eSheetDataInt = pd.ExcelFile(pivot_table)
                eSheetData = eSheetDataInt.parse(sheetCount)
            else:
                continue
    else:
        for s in dfs:
            m = re.search(fName, s) # case insensitive?
            if m:
                eSheetDataInt = pd.ExcelFile(pivot_table)
                eSheetData = eSheetDataInt.parse(sheetCount)
            else:
                continue
    return eSheetData

def openFileAsList(f):
    l = []
    with open(f, 'r') as fOpen:
        for i in fOpen:
            i = i.rstrip('\r\n')
            l.append(i)
    return l

def findColumnIdxStartStop(pdDF):
    xCt = -1
    xStart = -1
    xStop = -1
    foundInitialMatch = False
    for x in excelSheetName.columns.values:
        xCt += 1
        if xCt == 0:
            continue
        else:
            if xCt < 10 and foundInitialMatch == False: # column idx will not be greater than 9
                m = re.search('named', str(x))
                if m:
                    continue
                else:
                    foundInitialMatch = True
                    xStart = xCt
            else: 
                m = re.search('named', str(x))
                if m:
                    xStop = xCt
                    break
    return (xStart, xStop)

def parsePandasDfRows(col1ListFromPdDf):
    headers = True
    mamuA_indices = []
    mamuB_indices = []
    skipIndices = []
    genotypeList = []
    for idx,val in enumerate(col1ListFromPdDf):
        if headers:
            m = re.search('Comment', str(val))
            mA = re.search('Mamu-A', str(val))
            mB = re.search('Mamu-B', str(val))
            if m:
                headers = False
                skipIndices.append(idx)
            elif mA:
                mamuA_indices.append(idx)
            elif mB:
                mamuB_indices.append(idx)
            else:
                skipIndices.append(idx)
        else:
            m = re.search('Alleles', str(val))
            if m:
                skipIndices.append(idx)
            elif type(val) is float:
                skipIndices.append(idx)
            else:
                genotypeList.append(val)
    return (skipIndices, mamuA_indices, mamuB_indices, genotypeList)
'''
def dataToPandasOneHot(d, pdDf, idxList):
    if pdDf is None:
        pdDf = pd.DataFrame({'genotypeValues': idxList})
        pdDf.index = idxList
        pdDf.index.name = 'genotype'
        pdDfNew = pd.DataFrame.from_dict(d, orient='index').transpose()
        pdDf.index = idxList
        pdDf.index.name = 'genotype'
        alleleDFres = pd.concat([pdDf, pdDfNew], axis=1, join_axes=[pdDf.index])
        pdDf = alleleDFres
    else:
        alleleDFnew = pd.DataFrame.from_dict(d, orient='index').transpose()
        alleleDFnew.index = idxList
        alleleDFnew.index.name = 'genotype'
        alleleDFres = pd.concat([pdDf, alleleDFnew], axis=1, join_axes=[pdDf.index])
        pdDf = alleleDFres
    print('DataFrame is')
    print(pdDf)
    time.sleep(5)
    return pdDf

'''
def dataToPandasOneHot(d, pdDf, idxList):
    if pdDf is None:
        pdDf = pd.DataFrame.from_dict(d, orient='index').transpose()
        pdDf.index = idxList
        pdDf.index.name = 'genotype'
    else:
        alleleDFnew = pd.DataFrame.from_dict(d, orient='index').transpose()
        alleleDFnew.index = idxList
        alleleDFnew.index.name = 'genotype'
        alleleDFres = pd.concat([pdDf, alleleDFnew], axis=1, join_axes=[pdDf.index])
        pdDf = alleleDFres
    return pdDf

def parseIdxForMamuAMamuB(gList):
    # parses genotype names from filtered genotype list
    mamuA_nameIdxListCol1 = []
    mamuB_nameIdxListCol1 = []
    for idx, n in enumerate(gList):
        m_MamuA = re.search('Mamu_A', str(n))
        m_MamuB = re.search('Mamu_B', str(n))
        if m_MamuA:
            filterMamu = re.search('Mamu_AG', str(n))
            if filterMamu:
                continue
            else:
                mamuA_nameIdxListCol1.append(idx)
        if m_MamuB:
            mamuB_nameIdxListCol1.append(idx)
    return (mamuA_nameIdxListCol1, mamuB_nameIdxListCol1)

def parseGenotypeList(gList):
    r = []
    gListCheck = gList[0]
    gListCheckedString = gListCheck.split('_')
    gListItemToCheck = str(gListCheckedString[0])
    m = re.match('\d+', gListItemToCheck)
    if m:
        for g in gList:
            if type(g) is float:
                continue
            else:
                gString = g.split('_')
                gStringAsList = gString[1:]
                gStringAsString = '_'.join(gStringAsList)
                r.append(gStringAsString)
    else:
        return None
    return r
    
testFile = pd.ExcelFile(pivot_table)
readyToProceed = False
fName_none = ''
dataAsPandas = ''
excelSheetName = parseExcelWithPandas(fName_none, testFile)
processedPivotTable = False
if pivot_table:
    if excelSheetName.empty:
        print('The Excel sheet was found, but there was an error reading the Excel file.')
        print('Please do one of the following:\n\nProceed to the next cell and attempt to enter the sheet_name value,\n\nor\n\nExport the file as a csv, start from the beginning of the python notebook,\nchange the file name, and rerun all cells.\nBe sure to specify the file type as csv in the cell below when you run it.')
    else:
        print('Please examine the values that are listed from column 1 of the file.')
        print('If they are correct or what you would expect, click the next cell and press Run.')
        print('Otherwise, double check the filename and try again, or in the next cell,\nspecify the file type and the Excel sheet name, and then click Run.')
        print('\n\n')
        pdHeadersRowsCol1 = list(excelSheetName.iloc[1:10:,0])
        for h in pdHeadersRowsCol1:
            m = re.search('Animal ID', h)
            if m:
                processedPivotTable = True
            print(h + '\n')
        if processedPivotTable:
            print('Processed Pivot Table')
        else:
            print('raw pivot table')
        readyToProceed = True
else:
    print('No pivot table was found. Please do the following: \n1) check the filename and rerun the previous cell\n2) if you have already rerun the previous cell and are seeing this message again, \nproceed to the next cell and enter information for at least one of the following: ')
    print('\tfile_type: enter \'csv\' or \'excel\', depending on the file.')
    print('\tsheet_name: enter the sheet name for the excel spreadsheet or csv file')
    print('Then run the next two cells.')


Please examine the values that are listed from column 1 of the file.
If they are correct or what you would expect, click the next cell and press Run.
Otherwise, double check the filename and try again, or in the next cell,
specify the file type and the Excel sheet name, and then click Run.



Animal ID

# Reads Evaluated

# Reads Identified

% Unknown

Mamu-A Haplotype 1

Mamu-A Haplotype 2

Mamu-B Haplotype 1

Mamu-B Haplotype 2

Comments

Processed Pivot Table


In [12]:
#PRAGA MARK: structureData
# create data structure for input data, training data
def parseLabeledValues(l):
    r = []
    for i in l:
        iSplit = i.split('-')
        iSplitSorted = sorted(iSplit)
        iSplitSortdString = '-'.join(iSplitSorted)
        if iSplit[0] == iSplit[1]:
            r.append(iSplit[0])
        else:
            r.append(iSplitSortdString)
    return r

def parseDataIntoPandasDF(rStartInt, rStopInt, mamuA_rows, mamuB_rows, skipMamuRowInts, excelSheetName, gListForIndex):
    rRange = rStopInt - rStartInt
    mamuA_alleleList = []
    mamuB_alleleList = []
    alleleDF_MamuA = None
    alleleDF_MamuB = None
    for x in range(rStart, rStop):
        mamu_genotypes_oneHot = []
        dfValue = excelSheetName.iloc[:,x]
        # dfValueMerged = mergeDataFrameValues()
        dfValue_MamuA_1 = dfValue.iloc[int(mamuA_rows[0])]
        dfValue_MamuA_2 = dfValue.iloc[int(mamuA_rows[1])]
        dfValue_MamuB_1 = dfValue.iloc[int(mamuB_rows[0])]
        dfValue_MamuB_2 = dfValue.iloc[int(mamuB_rows[1])]
        pdDict_MamuA = dict()
        pdDict_MamuB = dict()
        additionalValuesBool = False
        for idx, row in dfValue.iteritems():
            if idx not in skipMamuRowInts:
                try:
                    if math.isnan(row):
                        mamu_genotypes_oneHot.append(0)
                        continue
                    else:
                        mamu_genotypes_oneHot.append(1)
                except TypeError:
                    mamu_genotypes_oneHot.append(1)
        pdDictKey_MamuA = str(dfValue_MamuA_1) + '-' + str(dfValue_MamuA_2)
        pdDictKey_MamuB = str(dfValue_MamuB_1) + '-' + str(dfValue_MamuB_2)
        pdDict_MamuA[pdDictKey_MamuA] = mamu_genotypes_oneHot
        pdDict_MamuB[pdDictKey_MamuB] = mamu_genotypes_oneHot
        alleleDF_MamuA = dataToPandasOneHot(pdDict_MamuA, alleleDF_MamuA, gListForIndex)
        alleleDF_MamuB = dataToPandasOneHot(pdDict_MamuB, alleleDF_MamuB, gListForIndex)
    return (alleleDF_MamuA, alleleDF_MamuB, gListForIndex)

def createTupleOfOneHotAndLabels(npDataFrame, rowRange):
    alleleDF_Mamu_parsedList = []
    alleleDF_Mamu_listLabels_Y = []
    rowRangeLimit = rowRange - 1
    # Developer Comment: JRC
    # I have no clue why, but this works, so I'm just going with it
    skippedRowCount = 0
    ct = 0
    for i in range(0, rowRangeLimit):
        # print('checking column ' + str(ct))
        # print('length of alleleDF_Mamu_parsedList is ' + str(len(alleleDF_Mamu_parsedList)))
        ct += 1
        dfColAsSeries = npDataFrame.iloc[:,i]
        if dfColAsSeries.name == 'nan-nan':
            skippedRowCount += 1
            continue
        dfColAsList = dfColAsSeries.tolist()
        alleleDF_Mamu_parsedList.append(dfColAsList)
        alleleDF_Mamu_listLabels_Y.append(dfColAsSeries.name)
    return (alleleDF_Mamu_parsedList, alleleDF_Mamu_listLabels_Y)

def parseOutGenotypeNames(parsedIndicesList, gListParsed):
    retList = []
    for a in parsedMamuAIndices:
        retList.append(gListParsed[a])
    return retList

def lambdaFunc(v):
    return int(v)

def openAndParse(f):
    listValue = []
    headerLine = ''
    start = True
    with open(f) as fOpen:
        for i in fOpen:
            if start:
                iLine = i.split(',')
                iLineDiscard = iLine.pop(0)
                headerLine = ','.join(iLine)
                start = False
            else:
                i = i.rstrip('\n')
                iSplit = i.split(',')
                iSplitInt = list(map(lambdaFunc, iSplit[1:]))
                # listValue.append((iSplit[0], iSplit[1:]))
                listValue.append((iSplit[0], iSplitInt))
    return (headerLine, listValue)

def trainingDataToDataObject(tList):
    # Developer Comment JRC:
    # initial data structure is
    # (genotypeList_string, [(haplotypeLabel_1, [row-one-hot-vector]),(haplotypeLabel_2, [row-one-hot-vector])])
    trainingDataList_Mamu = []
    colLabelsAsString = tList[0]
    colLabelsAsList = colLabelsAsString.split(',')
    tListValues = tList[1]
    rowListOfVectors = []
    genotypeList_trainingMamu = []
    for i in tListValues:
        rowListOfVectors.append(i[1])
        genotypeList_trainingMamu.append(i[0])
    for i in range(0, len(colLabelsAsList)):
        lol_entry = [colLabelsAsList[i], genotypeList_trainingMamu, [], rowListOfVectors, [], [], []]
        trainingDataList_Mamu.append(lol_entry)
    return trainingDataList_Mamu


def inputDataToDataObject(iListDF):
    inputDataList_Mamu = []
    listOfRowNames = []
    listOfRowValues = []
    listOfColumnNames = []
    for rIdx,rItem in iListDF.iterrows():
        listOfRowNames.append(str(rIdx))
        rowValues = list(rItem)
        listOfRowValues.append(rowValues)
    ct = 0
    for cItem in iListDF:
        colName = str(cItem)
        if colName == 'nan-nan':
            print('removing ' + str(colName))
            rmIdx = ct
            for rItem in listOfRowValues:
                del(rItem[rmIdx])
            ct += 1
            continue
        listOfColumnNames.append(colName)
        ct += 1
    for i in range(0, len(listOfColumnNames)):
        lol_entry = [listOfColumnNames[i], listOfRowNames, [], listOfRowValues, [], []]
        inputDataList_Mamu.append(lol_entry)
    return inputDataList_Mamu


# Developer Comment JRC
# The bug that kept skittering away was when doing a transform, pay careful attention to
# which rows and columns are used, and the way the new data structure is formed. 
# When doing a columnToRow transform, you must create an empty list of lists-or array of arrays-
# that has the same number of columns as the current dataset, then expand each list/array
# When doing a rowToColumn transform, you must create a single list/array, then scan the rows 
# for the appropriately indexed value, and then proceed across the rows
# As noted in other comments, in principle this will not change if the number of rows is expanded,
# however, again, careful attention must be paid to the size and structure of the resulting list/array
# since only specific lists/arrays in the newly expanded data object will have the correct count


def sliceNames(l):
    r = []
    for i in l:
        r.append(i[0])
    return r

def reorderTrainingAndInputDataLists(tDataList, iDataList):
    vertList_tData = tDataList[0][3]
    vertList_iData = iDataList[0][3]
    tDataGenotypeListNames = tDataList[0][1] # sync using order from trainingList
    iDataGenotypeListNames = iDataList[0][1]
    # sanity check
    if len(vertList_iData) != len(iDataGenotypeListNames):
        print('WARNING! length mismatch in iData.  Check data objects before proceeding.')
        print('length of ')
        print(len(vertList_iData))
        print(len(iDataGenotypeListNames))
        return None
    if len(vertList_tData) != len(tDataGenotypeListNames):
        print('WARNING! length mismatch in tData.  Check data objects before proceeding.')
        print(len(vertList_tData))
        print(len(tDataGenotypeListNames))
        return None
    tData_iter = iter(vertList_tData)
    iData_iter = iter(vertList_iData)
    padValue = len(iDataList[0][3][0])
    tDataTuple = list(zip(tDataGenotypeListNames, tData_iter))
    iDataTuple = list(zip(iDataGenotypeListNames, iData_iter))
    
    # Dev Comment JRC:
    # NOTE1: changing and padding the genotypes will not affect haplotype naming
    # AS LONG AS both data structures are in the format of rows -> genotypes, columns -> haplotypes
    # NOTE2: uncomment the lines below for debugging only
    updatediDataList = []
    for i in tDataGenotypeListNames:
        try:
            cIdx = iDataGenotypeListNames.index(i)
            # print('appending ')
            # print(iDataTuple[cIdx])
            checkedValue = iDataTuple[cIdx]
            updatediDataList.append(checkedValue)      
        except ValueError:
            # print(str(i) + ' not found!!')
            # print('appending')
            paddedList = [0]*padValue
            paddediData = (i, paddedList)
            # print(paddediData)
            updatediDataList.append(paddediData)
    updatediDataList_Genotypes, updatediDataList_onehotMultiList = zip(*updatediDataList)
    return (updatediDataList_Genotypes, updatediDataList_onehotMultiList)


def rows2Columns(l):
    c = []
    rowLen = len(l[0]) # assumes all rows have equal length
    for j in range(0, rowLen):
        singleRow = []
        for i in l:
            singleRow.append(i[j])
        c.append(singleRow)
    return c
    
def cols2Rows(l):
    r = []
    colLen = len(l[0])
    for i in range(0, colLen):
        singleColumn = []
        for j in l:
            singleColumn.append(j[i])
        r.append(singleColumn)
    return r

def transposeRowsAndColumns(pandasDF):
    rowNameList = []
    colListNames = []
    returnedDF = None
    pdRowList = []
    # create list of rownames, list of list of row values
    for rIdx,rItem in pandasDF.iterrows():
        rowNameList.append(str(rIdx))
        rowValues = list(rItem)
        pdRowList.append(rowValues)
    
    # Developer Comment JRC:
    # create list of columnNames, using column.values() 
    # is an alternative but use with caution, because it had a tendency to break functionality without warning    
    for cItem in pandasDF:
        colName = str(cItem)
        colListNames.append(colName)

    # transpose rows_columns to a columns_rows list, then create a transposed dataframe
    rowsColumns_xp_columnsRows = rows2Columns(pdRowList)
    pandasDF_haplotypeX_genotypeY = pd.DataFrame(rowsColumns_xp_columnsRows, index=colListNames, columns=indexList)
    return pandasDF_haplotypeX_genotypeY

def transposeRowsAndColumnsFromDataObject(dObject):
    # Dev Comment JRC: naming is in reference to the existing dObject, not the transformed object
    rowNameList = []
    colListNames = []
    rowListofRows = []
    rowNameList = dObject[0][1]
    rowListOfRows = dObject[0][3]
    transposed_rowsToColumns = rows2Columns(rowListOfRows)
    for i in range(0, len(dObject)):
        dObject[i][2] = transposed_rowsToColumns[i]
    return dObject


def parsedColumnData(l, idx):
    resList = []
    for itm in l:
        resList.append(itm[2][idx])
    return resList
def parseForML(l):
    labels_Y = []
    values_X = []
    for i in l:
        labels_Y.append(i[0])
        # tmp = parsedColumnData(l, ct)
        values_X.append(i[2])
    return (labels_Y, values_X)

In [6]:
# Developer Comment JRC:
'''
Given a table structure of values, where a horizontal axis is defined as x and a vertical axis is defined as y:
           Haplotype_1 | Haplotype_2 | Haplotype_3
genotype_1     0            0             1
genotype_2     0            1             0
genotype_3     1            1             0

len(x) may equal len(y), but it is not required
all x are equal length, and all y are equal length

let g = len(y)
let h = len(x)

a row-structured version of the data is defined as
A = [[genotype_1:[y_1_1, y_2_1,...,y_n_1]],[genotype_1:[y_1_2, y_2_2,...,y_n_2]]]
while a column-structured version of the data is defined as
B = [[Haplotype_1:[x_1_1, x_2_1,...,x_n_1],[Haplotype_2:[x_1_2, x_2_2,...,x_n_2]]]

transposing or transforming the values from row-structured to column-structured can be defined as:

transposing the X to the Y axis, defined as X_transposed == {X(row) and Y(column) values interchanged} and 
transposing the Y to the X axis, defined as Y_transposed == {Y(column) and X(row) values interchanged} can be described with:

len(X_transposed_column) == len(g)
len(Y_transposed_column) == len(h)


'''

'\nGiven a table structure of values, where a horizontal axis is defined as x and a vertical axis is defined as y:\n           Haplotype_1 | Haplotype_2 | Haplotype_3\ngenotype_1     0            0             1\ngenotype_2     0            1             0\ngenotype_3     1            1             0\n\nlen(x) may equal len(y), but it is not required\nall x are equal length, and all y are equal length\n\nlet g = len(y)\nlet h = len(x)\n\na row-structured version of the data is defined as\nA = [[genotype_1:[y_1_1, y_2_1,...,y_n_1]],[genotype_1:[y_1_2, y_2_2,...,y_n_2]]]\nwhile a column-structured version of the data is defined as\nB = [[Haplotype_1:[x_1_1, x_2_1,...,x_n_1],[Haplotype_2:[x_1_2, x_2_2,...,x_n_2]]]\n\ntransposing or transforming the values from row-structured to column-structured can be defined as:\n\ntransposing the X to the Y axis, defined as X_transposed == {X(row) and Y(column) values interchanged} and \ntransposing the Y to the X axis, defined as Y_transposed == {Y(co

In [13]:
#PRAGMA Mark: formatDataStructure
# format data for data structure, match data in input and training,
# then transpose to genotypes in the columns (gY) and haplotypes in the rows (hX)
v = findColumnIdxStartStop(excelSheetName)
if v[0] == -1:
    print('Error!  check dataframe!')
col1List = list(excelSheetName['Sample Sheet #']) # this needs to be updated
skipRows, mamuA_rows, mamuB_rows, genotypeList_unparsed = parsePandasDfRows(col1List)
genotypeList_parsed = []
genotypeList_parsed = parseGenotypeList(genotypeList_unparsed)

if genotypeList_parsed is None:
    genotypeList_parsed = genotypeList_unparsed


skipMamuRows = mamuA_rows + mamuB_rows + skipRows
skipMamuRows.sort()
parsedMamuIndices = parseIdxForMamuAMamuB(genotypeList_parsed)
parsedMamuAIndices = parsedMamuIndices[0] # indices matching "Mamu-A"
parsedMamuBIndices = parsedMamuIndices[1] # indices matching "Mamu-B"

rStart = v[0]
rStop = v[1] + 1
rRange = rStop - rStart
allele_DF_result = parseDataIntoPandasDF(rStart, rStop, mamuA_rows, mamuB_rows, skipMamuRows, excelSheetName, genotypeList_parsed)

alleleDF_MamuA_result = allele_DF_result[0]
alleleDF_MamuB_result = allele_DF_result[1]
alleleDF_MamuA_parsed = alleleDF_MamuA_result.iloc[parsedMamuAIndices,:]
alleleDF_MamuB_parsed = alleleDF_MamuB_result.iloc[parsedMamuBIndices,:]

alleleDF_MamuA_parsed_asDataObject = inputDataToDataObject(alleleDF_MamuA_parsed)
alleleDF_MamuB_parsed_asDataObject = inputDataToDataObject(alleleDF_MamuB_parsed)


# training data, testing here only tests the accuracy and is only for mhcA
trainingValues_mhcA = openAndParse('allele_df-trainingSet-HapA_transposed.csv')
testingValues_mhcA = openAndParse('allele_df-testingSet-HapA_transposed.csv')
trainingValues_mhcB = openAndParse('allele_df-trainingSet-HapB_transposed.csv')

trainingDataList_MamuA = trainingDataToDataObject(trainingValues_mhcA)
testingDataList_MamuA = trainingDataToDataObject(testingValues_mhcA)
trainingDataList_MamuB = trainingDataToDataObject(trainingValues_mhcB)


matched_inputDataListTuple_mhcA = reorderTrainingAndInputDataLists(trainingDataList_MamuA, alleleDF_MamuA_parsed_asDataObject)
matched_testingDataListTuple_mhcA = reorderTrainingAndInputDataLists(trainingDataList_MamuA, testingDataList_MamuA)
matched_inputDataListTuple_mhcB = reorderTrainingAndInputDataLists(trainingDataList_MamuB, alleleDF_MamuB_parsed_asDataObject)


updatedMatchedGenotypeList_testing = list(matched_testingDataListTuple_mhcA[0])
updatedMatchedOneHotVectorList_testing = list(matched_testingDataListTuple_mhcA[1])



for i in testingDataList_MamuA:
    i[1] = updatedMatchedGenotypeList_testing
    i[3] = updatedMatchedOneHotVectorList_testing

updatedMatchedGenotypeList = list(matched_inputDataListTuple_mhcA[0])
updatedMatchedOneHotVectorList = list(matched_inputDataListTuple_mhcA[1])
for i in alleleDF_MamuA_parsed_asDataObject:
    i[1] = updatedMatchedGenotypeList
    i[3] = updatedMatchedOneHotVectorList

updatedMatchedGenotypeList_mhcB = list(matched_inputDataListTuple_mhcB[0])
updatedMatchedOneHotVectorList_mhcB = list(matched_inputDataListTuple_mhcB[1])
for i in alleleDF_MamuB_parsed_asDataObject:
    i[1] = updatedMatchedGenotypeList_mhcB
    i[3] = updatedMatchedOneHotVectorList_mhcB

alleleDF_MamuA_hX_gY = transposeRowsAndColumnsFromDataObject(alleleDF_MamuA_parsed_asDataObject)
alleleDF_MamuA_hX_gY_training = transposeRowsAndColumnsFromDataObject(trainingDataList_MamuA)
alleleDF_MamuA_hX_gY_testing = transposeRowsAndColumnsFromDataObject(testingDataList_MamuA)
alleleDF_MamuB_hX_gY = transposeRowsAndColumnsFromDataObject(alleleDF_MamuB_parsed_asDataObject)
alleleDF_MamuB_hX_gY_training = transposeRowsAndColumnsFromDataObject(trainingDataList_MamuB)

trainingDataList_MamuB
# patch to remove newlines that slipped in, write a better one later
for i in alleleDF_MamuA_hX_gY_testing:
    iStrip = i[0].replace('\n', '')
    i[0] = iStrip
for i in alleleDF_MamuA_hX_gY_training:
    iStrip = i[0].replace('\n', '')
    i[0] = iStrip
# alleleDF_MamuB_hX_gY = transposeRowsAndColumns(alleleDF_MamuB_parsed)


removing nan-nan
removing nan-nan


In [ ]:
# Developer Comment JRC
'''
There is a fork in the workflow here.  Currently, the data is added to a pandas dataframe,
then the pandas dataframe is filtered for only MHC-A and MHC-B values using a pre-created index. 
Next, this filtered dataframe is used to create a transposed dataframe of rows and columns. 
An alternative option is one could create an unfiltered dataframe and transpose that, using the unparsed
genotypeList to verify the correct rows are selected, but all filtering *must* 
 - be completed before transposing row and column values
 - ONLY be done when genotypes are the row values
or the entire organization of the data structure will collapse.

'''

In [18]:
def valuesToIntList(valList):
    listedValues = []
    ct = 0
    l_strings = []
    l_ints = []
    l_strings_r = []
    l_ints_r = []
    npArrayList = []
    for i in valList:
        if i not in l_strings:
            l_strings_r.append(i)
            l_ints_r.append(ct)
            l_strings.append(i)
            l_ints.append(ct)
            npArrayList.append(ct)
            ct += 1
        else:
            v_i = l_strings.index(i)
            v_i_string = l_strings[v_i]
            v_ct = l_ints[v_i]
            l_strings_r.append(v_i_string)
            l_ints_r.append(v_ct)
            npArrayList.append(v_ct)
    npArrayTupleList = list(zip(l_strings_r,l_ints_r))
    return (npArrayList, npArrayTupleList)

def scanLabelTuple(s, t):
    for i in t:
        if i[0] == s:
            return (s, i[1])
    return (False, -1)
def updateLabelInts(valList, tupleList):
    listedValues = []
    ct = 0
    l_strings = []
    l_ints = []
    l_strings_r = []
    l_ints_r = []
    npArrayList = []
    for i in valList:
        if i not in l_strings:
            scannedResult = scanLabelTuple(i, tupleList)
            if scannedResult[0] != False:
                l_strings.append(scannedResult[0])
                l_ints.append(scannedResult[1])
                l_strings_r.append(scannedResult[0])
                l_ints_r.append(scannedResult[1])
                npArrayList.append(scannedResult[1])
            else:
                print('WARNING!  Value ' + str(i) + ' not found in training Data,\nwhich will cause a dramatic decrease in accuracy,\nbut this will be addressed in the next version')
                l_strings.append(scannedResult[0])
                l_ints.append(scannedResult[1])
                l_strings_r.append(scannedResult[0])
                l_ints_r.append(scannedResult[1])
                npArrayList.append(scannedResult[1])
        else:
            v_i = l_strings.index(i)
            v_i_string = l_strings[v_i]
            v_ct = l_ints[v_i]
            l_strings_r.append(v_i_string)
            l_ints_r.append(v_ct)
            npArrayList.append(v_ct)
    npArrayTupleList = list(zip(l_strings_r,l_ints_r))
    return (npArrayList, npArrayTupleList)



In [19]:
# MHC-A
training_set = parseForML(alleleDF_MamuA_hX_gY_training)
X_trainList = training_set[1]
Y_trainList = training_set[0]

# Developer Comment JRC: 
# uncomment the next two lines to use a premade testing dataset for debugging
# testing_set = parseForML(alleleDF_MamuA_hX_gY_testing)
# X_testList = testing_set[1]
# Y_testList = testing_set[0]

testing_set = parseForML(alleleDF_MamuA_hX_gY)
X_testList = testing_set[1]
Y_testListUnparsed = testing_set[0]
Y_testList = parseLabeledValues(Y_testListUnparsed)

X_train = np.array(X_trainList)
X_test = np.array(X_testList)
Y_trainTuple = valuesToIntList(Y_trainList)
Y_train = Y_trainTuple[0]

Y_testTuple = updateLabelInts(Y_testList, Y_trainTuple[1])
Y_test = Y_testTuple[0]


In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
forest_clf = RandomForestClassifier(random_state=42)
forest_clf.fit(X_train, Y_train)
forest_clf_pred = forest_clf.predict(X_test)

# Developer Comment JRC:
# uncomment the next two lines for a raw output
# print(forest_clf_pred)
# print(Y_test)
print('Accuracy Score is ')
print(accuracy_score(Y_test, forest_clf_pred))

def resultsLookup(rInt, tList):
    for t in tList:
        if t[1] == rInt:
            return t[0]
    return None
    
resultsList = []
for x in range(0, len(Y_test)):
    testValue = Y_test[x]
    predictedValue = forest_clf_pred[x]
    lookupString = resultsLookup(testValue, Y_trainTuple[1])
    lookupString_predicted = resultsLookup(predictedValue, Y_trainTuple[1])
    if testValue != predictedValue:
        print('The Classifier predicted ' + str(lookupString_predicted) + ' but the actual value was ' + str(lookupString))
        resultsList.append((lookupString_predicted, lookupString))
    else:
        resultsList.append((lookupString_predicted, lookupString))


s_out_predicted = ''
s_out_actual = ''
for x in resultsList:
    s_out_predicted += str(x[0]) + ','
    s_out_actual += str(x[1]) + ','
s_out_predicted = s_out_predicted[:-1]
s_out_actual = s_out_actual[:-1]
with open('resultsMHC_A_Output.txt', 'w') as fWrite:
    # print("Purchase Amount: {}".format(TotalAmount), file=text_file)
    print('Table of Predicted (row 1) vs. Actual values (row 2)', file=fWrite)
    # print('\n', file=fWrite)
    print(s_out_predicted, file=fWrite)
    # print('\n', file=fWrite)
    print(s_out_actual, file=fWrite)
    print('\n', file=fWrite)
    print('Accuracy Score is: ' + str(accuracy_score(Y_test, forest_clf_pred)),file=fWrite)
    
print('\n\nThe output has been written to the file resultsMHC_A_Output.txt.csv')

Accuracy Score is 
0.959459459459
The Classifier predicted A023 but the actual value was A016-A023
The Classifier predicted A004 but the actual value was A004-A025
The Classifier predicted A025 but the actual value was A011-A025


The output has been written to the file resultsMHC_A_Output.txt


In [21]:
# MHC-B
training_set = parseForML(alleleDF_MamuB_hX_gY_training)
X_trainList = training_set[1]
Y_trainList = training_set[0]

testing_set = parseForML(alleleDF_MamuB_hX_gY)
X_testList = testing_set[1]
Y_testListUnparsed = testing_set[0]
Y_testList = parseLabeledValues(Y_testListUnparsed)

X_train = np.array(X_trainList)
X_test = np.array(X_testList)
Y_trainTuple = valuesToIntList(Y_trainList)
Y_train = Y_trainTuple[0]

Y_testTuple = updateLabelInts(Y_testList, Y_trainTuple[1])
Y_test = Y_testTuple[0]

WARNING!  Value B012b-B093' not found in training Data,
which will cause a dramatic decrease in accuracy,
but this will be addressed in the next version
WARNING!  Value B024a-B106' not found in training Data,
which will cause a dramatic decrease in accuracy,
but this will be addressed in the next version


In [22]:
forest_clf = RandomForestClassifier(random_state=42)
forest_clf.fit(X_train, Y_train)
forest_clf_pred = forest_clf.predict(X_test)

# Developer Comment JRC:
# uncomment the next two lines for a raw output
# print(forest_clf_pred)
# print(Y_test)
print('Accuracy Score is ')
print(accuracy_score(Y_test, forest_clf_pred))

def resultsLookup(rInt, tList):
    for t in tList:
        if t[1] == rInt:
            return t[0]
    return None
    
resultsList = []
for x in range(0, len(Y_test)):
    testValue = Y_test[x]
    predictedValue = forest_clf_pred[x]
    lookupString = resultsLookup(testValue, Y_trainTuple[1])
    lookupString_predicted = resultsLookup(predictedValue, Y_trainTuple[1])
    if testValue != predictedValue:
        print('The Classifier predicted ' + str(lookupString_predicted) + ' but the actual value was ' + str(lookupString))
        resultsList.append((lookupString_predicted, lookupString))
    else:
        resultsList.append((lookupString_predicted, lookupString))


s_out_predicted = ''
s_out_actual = ''
for x in resultsList:
    s_out_predicted += str(x[0]) + ','
    s_out_actual += str(x[1]) + ','
s_out_predicted = s_out_predicted[:-1]
s_out_actual = s_out_actual[:-1]
with open('resultsMHC_B_Output.txt', 'w') as fWrite:
    # print("Purchase Amount: {}".format(TotalAmount), file=text_file)
    print('Table of Predicted (row 1) vs. Actual values (row 2)', file=fWrite)
    # print('\n', file=fWrite)
    print(s_out_predicted, file=fWrite)
    # print('\n', file=fWrite)
    print(s_out_actual, file=fWrite)
    print('\n', file=fWrite)
    print('Accuracy Score is: ' + str(accuracy_score(Y_test, forest_clf_pred)),file=fWrite)
    
print('\n\nThe output has been written to the file resultsMHC_B_Output.txt.csv')

Accuracy Score is 
0.932432432432
The Classifier predicted B012b-B093 but the actual value was None
The Classifier predicted B015a-B012b but the actual value was B012b-B015a
The Classifier predicted B015a-B012b but the actual value was B012b-B015a
The Classifier predicted B015a-B012b but the actual value was B012b-B015a
The Classifier predicted B024a-B106 but the actual value was None


The output has been written to the file resultsMHC_B_Output.txt
